# **EXPLOITATION ZONE**

In [1]:
 !git clone https://github.com/OscarMoliina/betterlifebetterhealth.git

Cloning into 'betterlifebetterhealth'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 125 (delta 17), reused 29 (delta 11), pack-reused 87
Receiving objects: 100% (125/125), 82.53 MiB | 23.84 MiB/s, done.
Resolving deltas: 100% (45/45), done.
Updating files: 100% (31/31), done.


In [2]:
!pip3 install pyspark
!pip3 install duckdb
import pyspark

from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, mean, when, lit, count
from pyspark.sql.window import Window
from pyspark.sql.functions import mean
from pyspark.sql.functions import mean
from pyspark.sql.types import IntegerType, DoubleType

import pandas as pd
import matplotlib.pyplot as plt
import duckdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=d67a3fe67aea9ea34b7c0e83f92257cd0e041c0944cabbdd728c76feff643026
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


**Connexió:**

In [4]:
conn = duckdb.connect("/content/betterlifebetterhealth/src/trusted_zone.db")

In [3]:
spark = SparkSession.builder \
    .appName("Preprocessing") \
    .config("spark.jars", "/content/betterlifebetterhealth/src/duckdb.jar") \
    .getOrCreate()

dem = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:/content/betterlifebetterhealth/src/trusted_zone.db") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .option("dbtable", "dem") \
    .load()

mh = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:/content/betterlifebetterhealth/src/trusted_zone.db") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .option("dbtable", "mh") \
    .load()

soc = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:/content/betterlifebetterhealth/src/trusted_zone.db") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .option("dbtable", "soc") \
    .load()

**RECONCILIACIÓ DE LES DADES**

In [9]:
import requests

def get_wikidata_id(country_name):
    url = 'https://www.wikidata.org/w/api.php'
    params = {
        'action': 'wbsearchentities',
        'language': 'en',
        'format': 'json',
        'search': country_name,
        'type': 'item'
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        results = response.json().get('search')
        if results:
            for result in results:
                if result.get('label').lower() == country_name.lower():
                    return result['id']
    return None

dem_pd = dem.toPandas()
soc_pd = soc.toPandas()
mh_pd = mh.toPandas()

dem_pd['Country'].apply(get_wikidata_id)
soc_pd['Country'].apply(get_wikidata_id)
mh_pd['Country'].apply(get_wikidata_id)

dem = dem_pd.to_sql('dem', conn, if_exists='fail')
soc = soc_pd.to_sql('soc', conn, if_exists='fail')
mh = mh_pd.to_sql('mh', conn, if_exists='fail')

Py4JJavaError: An error occurred while calling o33.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (c65edcd6ee07 executor driver): java.sql.SQLException: IO Error: Could not set lock on file "/content/betterlifebetterhealth/src/trusted_zone.db": Conflicting lock is held in /usr/bin/python3.10 (PID 418). See also https://duckdb.org/docs/connect/concurrency
	at org.duckdb.DuckDBNative.duckdb_jdbc_startup(Native Method)
	at org.duckdb.DuckDBConnection.newConnection(DuckDBConnection.java:48)
	at org.duckdb.DuckDBDriver.connect(DuckDBDriver.java:41)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(ConnectionProvider.scala:102)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConn

In [ ]:
paisos_soc = soc.select('Country').distinct()
paisos_mh = mh.select('Country').distinct()
paisos_dem = dem.select('Country').distinct()

intersecció_2 = paisos_soc.intersect(paisos_mh).intersect(paisos_dem).orderBy('Country')
paisos_coincidents = [row['Country'] for row in intersecció_2.collect()]
print(paisos_coincidents)

dem = dem.filter(dem['Country'].isin(paisos_coincidents))
soc = soc.filter(soc['Country'].isin(paisos_coincidents))
mh = mh.filter(mh['Country'].isin(paisos_coincidents))

In [ ]:
dem.write \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:exploitation_zone.db") \
    .option("dbtable", "dem_reconciliated") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .save()

soc.write \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:exploitation_zone.db") \
    .option("dbtable", "soc_reconciliated") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .save()

mh.write \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:exploitation_zone.db") \
    .option("dbtable", "mh_reconciliated") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .save()

**JOIN**

In [5]:
mh.columns

['Country',
 'Year',
 'Schizophrenia (%)',
 'Bipolar disorder (%)',
 'Eating disorders (%)',
 'Anxiety disorders (%)',
 'Drug use disorders (%)',
 'Depression (%)',
 'Alcohol use disorders (%)']

In [6]:
dem.columns

['Country',
 'Year',
 'Area_Km2',
 'CBR',
 'CDR',
 'Deaths',
 'E0',
 'Medage',
 'MR0_4',
 'Pop_Dens',
 'GSCA']

In [7]:
soc.columns

['Country',
 'Year',
 'Total population',
 'Total population, male (%)',
 'Total population, female (%)',
 'Mean age of women at birth of first child',
 'Women in the Labour Force, Percent of corresponding total for both sexes',
 'Female tertiary students, percent of total',
 'Female legislators, senior officials and managers, percent of total',
 'Female professionals, percent of total for both sexes',
 'Female clerks, percent of total for both sexes',
 'Female craft and related workers, percent of total for both sexes',
 'Female plant and machine operators and assemblers, percent of total for both sexes',
 'Female members of parliament, percent of total',
 'Total employment, growth rate',
 'Unemployment rate',
 'Youth unemployment rate',
 'GDP at current prices and PPPs, millions of US$',
 'GDP per capita at current prices and PPPs, US$',
 'Final consumption expenditure per capita, US Dollars, current PPPs',
 'Purchasing power parity (PPP), NCU per US$',
 'Consumer price index, growth

Mapa Europa o del Mon relacionant:

1. (DEM AMB MH)qualevol malaltia amb Pop_Dens
2. Index dona sense relacionar-ho amb cap altre taula.
3. (SOC AMB MH) qualevol malaltia amb Unemployment rate

In [15]:
# Realiza el primer join y selecciona las columnas deseadas, renombrando para evitar duplicados si es necesario
joined_data = dem.join(mh, (dem["Country"] == mh["Country"]) & (dem["Year"] == mh["Year"]), "inner")
selected_columns = [dem[col] for col in dem.columns] + [mh[col] for col in mh.columns if col not in ["Country", "Year"]]
first_join = joined_data.select(*selected_columns)

# Realiza el segundo join y selecciona las columnas deseadas, evitando duplicados
final_join = first_join.join(soc, (first_join["Country"] == soc["Country"]) & (first_join["Year"] == soc["Year"]), "inner")
selected_columns_final = [first_join[col] for col in first_join.columns] + [soc[col] for col in soc.columns if col not in ["Country", "Year"]]
result = final_join.select(*selected_columns_final)


In [21]:
result.columns

['Country',
 'Year',
 'Area_Km2',
 'CBR',
 'CDR',
 'Deaths',
 'E0',
 'Medage',
 'MR0_4',
 'Pop_Dens',
 'GSCA',
 'Schizophrenia (%)',
 'Bipolar disorder (%)',
 'Eating disorders (%)',
 'Anxiety disorders (%)',
 'Drug use disorders (%)',
 'Depression (%)',
 'Alcohol use disorders (%)',
 'Total population',
 'Total population, male (%)',
 'Total population, female (%)',
 'Mean age of women at birth of first child',
 'Women in the Labour Force, Percent of corresponding total for both sexes',
 'Female tertiary students, percent of total',
 'Female legislators, senior officials and managers, percent of total',
 'Female professionals, percent of total for both sexes',
 'Female clerks, percent of total for both sexes',
 'Female craft and related workers, percent of total for both sexes',
 'Female plant and machine operators and assemblers, percent of total for both sexes',
 'Female members of parliament, percent of total',
 'Total employment, growth rate',
 'Unemployment rate',
 'Youth unemplo

In [24]:
df['Country']

2038         Turkey
144     Switzerland
145     Switzerland
146     Switzerland
147     Switzerland
           ...     
2990    Netherlands
1047       Slovenia
1878         Norway
1876         Norway
2276        Iceland
Name: Country, Length: 3264, dtype: object

In [27]:
import plotly.express as px
import pandas as pd

df = result.toPandas()
df.sort_values('Year', inplace=True)

european_countries = [
    'Germany', 'France', 'Italy', 'Spain', 'United Kingdom',
    'Poland', 'Romania', 'Netherlands', 'Belgium', 'Greece',
    'Czech Republic', 'Portugal', 'Sweden', 'Hungary', 'Austria',
    'Bulgaria', 'Denmark', 'Finland', 'Slovakia', 'Ireland',
    'Croatia', 'Lithuania', 'Slovenia', 'Latvia', 'Estonia',
    'Cyprus', 'Luxembourg', 'Malta'
]

df_europe = df[df['Country'].isin(european_countries)]


# Definir los límites para la escala de colores
min_schizophrenia = df_europe['GDP at current prices and PPPs, millions of US$'].min()
max_schizophrenia = df_europe['GDP at current prices and PPPs, millions of US$'].max()

# Crea el gráfico
fig = px.choropleth(df,
                    locations="Country",
                    locationmode='country names',
                    color='GDP at current prices and PPPs, millions of US$',
                    hover_name="Country",
                    animation_frame="Year",
                    color_continuous_scale=px.colors.sequential.Plasma,
                    title='Evolución de la Anxiety en Europa por año',
                    range_color=[min_schizophrenia, max_schizophrenia])

fig.update_layout(geo=dict(scope='europe'))
fig.show()
